# Chart Descriptions

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/andrewm4894/netdata-gpt-notebooks/blob/main/notebooks/chart_descriptions/chart_descriptions.ipynb)

In [2984]:
# if running in colab uncomment the following line and run it to install the required packages
#!pip install python-dotenv netdata-pandas openai

In [2985]:
import os
from dotenv import load_dotenv
import pandas as pd
import numpy as np
from netdata_pandas.data_cloud import get_data_cloud
import openai
import pprint as pp
from urllib.parse import urlparse
import requests
import json
from datetime import datetime

# load tokens from .env file
load_dotenv()

NETDATA_API_TOKEN = os.getenv('NETDATA_API_TOKEN')
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')

openai.api_key = OPENAI_API_KEY

In [2986]:
# inputs
space_id = 'ea93d7b8-0df6-45c0-b13d-1560996c89eb' 
room_id = 'd8a4e0c5-7c79-4145-900e-83a9f06fcb6a'
chart = None
output_dir = 'output'

now = datetime.now().strftime("%Y%m%d_%H%M%S")

In [2987]:
def get_charts_cloud(space_id, room_id, api_token=None, base_url='https://app.netdata.cloud', node_ids=[]):
    """Get charts from netdata cloud api.
    """
    
    if api_token is None:
        api_token = os.getenv('NETDATA_API_TOKEN')
    
    base_url = 'https://app.netdata.cloud'
    url = f'{base_url}/api/v2/spaces/{space_id}/rooms/{room_id}/charts'
    headers = {'Accept': '*/*', 'Content-Type': 'application/json', 'Authorization': f'Bearer {api_token}'}
    data = {
        'filter': {
            'nodeIDs': node_ids,
        }
    }
    r = requests.post(url, headers=headers, data=json.dumps(data))
    
    if r.status_code != 200:
        
        print(f'Error: {r.status_code, r.text}')
        return None
    
    else:
            
        return r.json()['results']


charts = get_charts_cloud(space_id, room_id, api_token=NETDATA_API_TOKEN)

In [2988]:
if chart == None:
    charts_list = list(charts.keys())
    #charts_list = [c for c in charts_list if c.startswith('system.')]
    charts_list = [c for c in charts_list if 'prometheus.' not in c]
    charts_list = [c for c in charts_list if 'netdata.' not in c]
    chart = np.random.choice(charts_list)

print(chart)

httpcheck.status


In [2989]:
df = get_data_cloud(space_id, room_id, chart)
print(df.shape)
print(df.head())

dimensions = df.columns
print(dimensions)

(13, 2)
                     bad_content  success
time                                     
2023-04-03 16:20:57            1        2
2023-04-03 16:21:00            1        2
2023-04-03 16:21:03            1        2
2023-04-03 16:21:06            1        2
2023-04-03 16:21:09            1        2
Index(['bad_content', 'success'], dtype='object')


In [2990]:
chart_json = {}
chart_json['id'] = chart
chart_json['title'] = charts[chart]['title']
chart_json['dimensions'] = list(dimensions)
chart_json['units'] = charts[chart]['units']
chart_json['family'] = charts[chart]['family']
chart_json['context'] = charts[chart]['context']
chart_json['chart_type'] = charts[chart]['chartType']
chart_json['chart_labels'] = charts[chart]['chartLabels']
chart_json

{'id': 'httpcheck.status',
 'title': 'HTTP Check Status',
 'dimensions': ['bad_content', 'success'],
 'units': 'boolean',
 'family': 'status',
 'context': 'httpcheck.status',
 'chart_type': 'line',
 'chart_labels': None}

In [2991]:
prompt = f"""
You are an experienced SRE and sysadmin.

You are monitoring your infrastructure using Netdata Cloud.

You are documenting individual charts and their dimensions to help other users.

The chart_json object is available to you and follows the format below:

```json
{{
    "id": "the chart id",
    "title": "the chart title",
    "dimensions": "a list of the dimensions",
    "units": "the units of the chart",
    "family": "the menu family of the chart",
    "context": "context of the chat. it follows structure like <type>.<name> where <type> also impacts where in the menu the chart appears and so can give hints of what it might relate to",
    "chart_type": "usually line or stacked or area - this is the type of chart",
    "chart_labels": "some optional labels or tags for the chart that cone also sometimes be useful to help understand the chart",
}}
```

Here is the individual chart_json object describing the `{chart}` chart you are documenting:

```json
{chart_json}
```

Can you write short, useful and educational description of the chart and its dimensions? 

Please follow a json format like this (the output needs to be valid json):

```json
{{
    "chart_id": "{chart}",
    "chart_description": "<add description here>",
    "dimension_descriptions": [
        {{
            "<dimension name>": "<add dimension description here>",
            ...
        }}]
}}
```
"""
print(prompt)


You are an experienced SRE and sysadmin.

You are monitoring your infrastructure using Netdata Cloud.

You are documenting individual charts and their dimensions to help other users.

The chart_json object is available to you and follows the format below:

```json
{
    "id": "the chart id",
    "title": "the chart title",
    "dimensions": "a list of the dimensions",
    "units": "the units of the chart",
    "family": "the menu family of the chart",
    "context": "context of the chat. it follows structure like <type>.<name> where <type> also impacts where in the menu the chart appears and so can give hints of what it might relate to",
    "chart_type": "usually line or stacked or area - this is the type of chart",
    "chart_labels": "some optional labels or tags for the chart that cone also sometimes be useful to help understand the chart",
}
```

Here is the individual chart_json object describing the `httpcheck.status` chart you are documenting:

```json
{'id': 'httpcheck.status

In [2992]:
# build messages list to pass to openai
messages=[
    {"role": "user", "content": prompt}
]

In [2993]:
# call openai api
completion = openai.ChatCompletion.create(
  model="gpt-3.5-turbo",
  messages=messages,
)

In [2994]:
reply_content = completion.choices[0].message.content
#pp.pprint(reply_content)
reply_content = reply_content.replace('",\n        }]','"\n        }]')
print(reply_content)

{
    "chart_id": "httpcheck.status",
    "chart_description": "The HTTP Check Status chart shows the status of the last HTTP health check. If the check is successful, it will display as True, if the check is unsuccessful, it will display as False. This chart allows you to monitor the health of your HTTP services.",
    "dimension_descriptions": [
        {
            "bad_content": "This dimension shows if the HTTP health check found bad content on the page. If True, it means that the check found unexpected or incorrect content on the page.",
            "success": "This dimension shows if the HTTP health check was successful. If True, it means that the check was able to connect to the web server and receive a response."
        }
    ]
}


In [2995]:
chart_description_json = json.loads(reply_content)

In [2996]:
file_name_prompt = f'{chart}__{now}_PROMPT.txt'
with open(f'{output_dir}/{file_name_prompt}', 'w') as f:
    f.write(prompt)

In [2997]:
file_name_result = f'{chart}__{now}_RESULT.json'
with open(f'{output_dir}/{file_name_result}', 'w') as f:
    json.dump(chart_description_json, f, indent=4)